In [ ]:
api_key = 'abcde' #this key does not work anymore
endpoint = 'fixtures'
params = {
    'league' : 39,
    'season' : 2010
}

Downloading Data

In [18]:
import requests
import json
def download_content(api_key, params, endpoint, output_name, output_folder_path=''):
    req = requests.get(
        url=f' https://v3.football.api-sports.io/{endpoint}',
        params = params,
        headers= {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': api_key
        }
    )

    content = str(req.content)
    content = content[2:]
    content = content[:-1]
    content = content.replace('\\','')



    #save response as a JSON file
    with open(f'{output_folder_path}{output_name}.json','w') as outfile:
        outfile.write(content)

def fetch_content(api_key, params, endpoint):
    req = requests.get(
        url=f' https://v3.football.api-sports.io/{endpoint}',
        params = params,
        headers= {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': api_key
        }
    )

    content = str(req.content)
    content = content[2:]
    content = content[:-1]
    content = content.replace('\\','')


    #save response as a JSON file
    with open(f'temp.json','w') as outfile:
        outfile.write(content)

    with open(f'temp.json') as f:
        data = json.load(f)

    data = data['response']

    return data
    

In [19]:
def beautify_json(filename,path=''):
    data = None
    with open(f'{path}{filename}.json') as f:
        data = json.load(f)

    data = data['response']
    print(data[0])
    json_string = json.dumps(data, indent=4) 

    with open(f'{path}{filename}.json','w') as outfile:
        outfile.write(json_string)
    


Parsing The JSON To Access The Data

Getting the Fixtures IDs and Match Stats of all fixtures in all completed seasons for a given league

In [ ]:
import json
filename = 'other_data/leagues'
ids_to_find = [253] #league ids

with open(f'{filename}.json') as f:
    data = json.load(f)

for id in ids_to_find:
    years = [] #list of years to extract data from
    
    for league in data:
        if league['league']['id'] == id: #found league

            #print seasons
            for s in league['seasons']:
                #see if they have match statistics
                if s['coverage']['fixtures']['statistics_fixtures'] and s['current'] == False:
                    years.append(s['year'])

    #Download all applicable years
    for year in years:

        download_content(api_key = 'xxxxx', 
                        params = {
                        'league' : id,
                        'season' : year
                    },
                        endpoint = 'fixtures',
                        output_name = f'fixtures_{id}_{year}', 
                        output_folder_path=f'other_leagues/fixtures_{id}/')
        
        beautify_json(filename=f'fixtures_{id}_{year}',path=f'other_leagues/fixtures_{id}/')

        with open(f'other_leagues/fixtures_{id}/fixtures_{id}_{year}.json') as f:
            data = json.load(f)
    
        content_to_file = []

        for fixture in data:

            
            fixture_id = fixture['fixture']['id']
            timestamp = fixture['fixture']['timestamp']
            home_team = fixture['teams']['home']['name']
            home_team_id = fixture['teams']['home']['id']
            away_team = fixture['teams']['away']['name']
            away_team_id =fixture['teams']['away']['id']

            try:
                match_stats = fetch_content(
                    api_key='6e7203791e94328d0a5f014d63969180',
                    params = {
                            'fixture':fixture_id
                        },
                        endpoint = 'fixtures/statistics'
                )

                home_stats = match_stats[0]
                away_stats = match_stats[1]

                to_save =  {
                    'fixture_id' : fixture_id,
                    'timestamp' : timestamp,
                    'home_team' : home_team,
                    'home_team_id' : home_team_id,
                    'away_team' : away_team,
                    'away_team_id' : away_team_id,
                    'home_stats' : home_stats,
                    'away_stats': away_stats
                }

                content_to_file.append(to_save)

            except:
                continue

        
        json_string = json.dumps(content_to_file, indent=4) 

        with open(f'other_leagues/{id}_stats/{id}_{year}.json','w') as outfile:
            outfile.write(json_string)
    

{'fixture': {'id': 491123, 'referee': 'Jair Marrufo, USA', 'timezone': 'UTC', 'date': '2015-03-07T03:00:00+00:00', 'timestamp': 1425697200, 'periods': {'first': 1425697200, 'second': 1425700800}, 'venue': {'id': None, 'name': 'StubHub Center', 'city': 'Los Angeles'}, 'status': {'long': 'Match Finished', 'short': 'FT', 'elapsed': 90, 'extra': None}}, 'league': {'id': 253, 'name': 'Major League Soccer', 'country': 'USA', 'logo': 'https://media.api-sports.io/football/leagues/253.png', 'flag': 'https://media.api-sports.io/flags/us.svg', 'season': 2015, 'round': 'Regular Season - 1', 'standings': False}, 'teams': {'home': {'id': 1605, 'name': 'Los Angeles Galaxy', 'logo': 'https://media.api-sports.io/football/teams/1605.png', 'winner': True}, 'away': {'id': 1607, 'name': 'Chicago Fire', 'logo': 'https://media.api-sports.io/football/teams/1607.png', 'winner': False}}, 'goals': {'home': 2, 'away': 0}, 'score': {'halftime': {'home': 0, 'away': 0}, 'fulltime': {'home': 2, 'away': 0}, 'extratime